In [59]:
import requests
from bs4 import BeautifulSoup as bs
import os
import pickle
import numpy as np
import time

In [47]:
def take_n_urls(n):

  main_url = "https://myanimelist.net/topanime.php"

  # this list will contain all the urls we'll retrieve
  urls = [] 

  # each page shows 50 elements and we can retrieve each page by manipulating the "limit" query
  for limit in range(0, n, 50): 
    content = requests.get(main_url,
                           params={"limit": limit})
    if content.status_code == 404:
      print(f"Page with limit {limit} was not found. Interrumpting and returning pages found")
      break
    soup = bs(content.content, "html.parser")

    # from the content of each page we retrieve the portions that contain the urls
    results = soup.find_all("a", 
                            class_= "hoverinfo_trigger fl-l ml12 mr8")

    # finally, we take the string containing each url by taking the attribute href,
    # and we append them in the urls list
    for result in results:
      url = result["href"]
      if url not in urls:  # check for duplicates
          urls.append(url)

  return urls

In [48]:
if "urls.txt" not in os.listdir():
    urls = take_n_urls(20000)
    # Since the output of this step has to be a txt file, here we write one with each
    # url separated by a newline
    with open("urls.txt", "w") as file:
      file.write("\n".join(urls_str))
else:
    with open("urls.txt", "r") as file:
        print("Loading urls...")
        urls = file.read().split("\n")
        print("Done!")

Loading urls...
Done!


In [49]:
# we end up with 19131 urls. 
# I added a check that tells us when we have exceeded the length of the ranking list and returns what has been found
# up until that moment (so to avoid losing any more time with get requests that point to nothing)
# I know in the assignment they said 20000 but I'm fairly sure that's all the entries. 
# This is easy to see if we manually set the limit in the url and check the results. 
# For example: https://myanimelist.net/topanime.php?limit=15000 contains rankings 15001-15050. The first entry is
# Big X Episode 0. If we check our list with urls[15000] (remember that our list is 0-indexed) we obtain the same result.
# This to me seems to point to a correct behavior from the function, but let me know what you think.

print(len(urls)) 
print(urls[15000])
urls_str = list(map(str, urls))

19130
https://myanimelist.net/anime/30839/Big_X_Episode_0


In [54]:
# Here we create the directory where the html pages will be stored
if "html_pages" not in os.listdir():
    os.mkdir("html_pages")

In [62]:
if "counter_pages" not in os.listdir():
    start = 0
else:
    with open("counter_pages", "rb") as counter_file:
        start = pickle.load(counter_file) + 1

print(f"Starting from anime #{start}")
n = len(urls)
for i in range(start, n):
    ranking_page = str(int(np.floor(i/50)))
    if i % 50 == 0 or "ranking_page_{ranking_page}" not in os.listdir("./html_pages"):
        time.sleep(10)
        if "ranking_page_{ranking_page}" not in os.listdir("./html_pages"):
            os.mkdir(f"html_pages/ranking_page_{ranking_page}")
    html_page = requests.get(urls[i])
    with open (f"html_pages/ranking_page_{ranking_page}/article_{i}.html", "w") as file:
        file.write(html_page.text)
    with open ("counter_pages", "wb") as counter_file:
        pickle.dump(i, counter_file)


Starting from anime #121


KeyboardInterrupt: 

## 5. Algorithmic question
You consult for a personal trainer who has a *back-to-back sequence* of requests for appointments. A sequence of requests is of the form
    > 30, 40, 25, 50, 30, 20
where each number is the time that the person who makes the appointment wants to spend.
You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, `[30, 50, 20]` is an acceptable solution (of duration *100*), but `[30, 40, 50, 20]` is not, because *30* and *40* are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration *110*.
1. Write an algorithm that computes the acceptable solution with the longest possible duration.
2. Implement a program that given in input an instance in the form given above, gives the optimal solution.


In [30]:
import numpy as np

### 1. Write an algorithm that computes the acceptable solution with the longest possible duration.

##### Here we consider that all values in the instance are unique and len(instance) = n

To compute the acceptable solution with the longest possible duration, we have to follow several steps :
1. Compute every possible solution : So for that, list all sublists which represent each possible list of appointments.
2. For each sublist, tell if it is acceptable or not, so if there are two consecutive appointments or not.
3. Compute the total duration of each acceptable solution.
4. Finally, return the solution which correspond to the maximum duration.

```
Input: 
    instance: list of length n

function optimal_solution(instance):
    n=len(instance)
    for i=0 to n: 
        sublists = sublists + [all sublists with i elements]
    
    acceptable_solutions=[all element of sublists which are acceptable]
    
    durations = [duration of each element of acceptable_solutions]
    max_duration = max(durations)
    optimal_solutions = [sublists of instance with total duration == max_durations]
    
    return optimal_solutions, max_duration
```

### 2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

First of all, we create a function ```is_acceptable(solution, instance)``` that says if a solution is acceptable or not, i.e. there is not two consecutive requests of the instance in the solution.

Then, the ```longest_acceptable_duration(instance)``` compute all possible sublists of the instance, test it with the function ```is_acceptable(solution, instance)```, sum every acceptable list (to compute the duration of each acceptable solution) and return the maximum. 

In [142]:
def is_acceptable(solution, instance):
    res=True 
    for x,y in zip(solution[:-1],solution[1:]):
        i= instance.index(x)
        #index1 = np.where(np.array(instance) == x)
        #for i in index[0]:
        #if the next element is y 
        if instance[i+1] == y:
            res = False
    return res

def optimal_solution(instance):
    sublists=[]
    
    for i in range(1, len(instance)+1):
        sublists+=[list(x) for x in combinations(instance, i)]
        
    mask = [is_acceptable(solution, instance) for solution in sublists]
    acceptable_sol = np.array(sublists)[mask]
    
    durations = [sum(L) for L in acceptable_sol]
    max_duration = max(durations)
    
    index_optimal_solutions = np.where(np.array(durations)==max_duration)
    
    return list(np.array(acceptable_sol)[index_optimal_solutions]), max_duration

In [143]:
instance = [30, 40, 25, 50, 30, 20]
optimal_solutions, max_duration = optimal_solution(instance)
print(optimal_solutions, max_duration)

[[40, 50, 20]] 110
